# Tools / Function Call

Here we follow [this tutorial](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/05%20Agents%20with%20Tool).

## TRPG Agent for Dungeons & Dragons (DnD)

In [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/05%20Agents%20with%20Tool), we create two agents (Dungeon Master - DM, and Player) to play a Dungeons & Dragons TRPG game. Essentially, there is a tool for dice rolling. When necessary, the DM needs to decide whether or not to use this tool to specify what happens in the future.

For more information about how DnD is played, check the [Wiki page](https://en.wikipedia.org/wiki/Dungeons_%26_Dragons).

In [1]:
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
import random
import json
from abc import ABC, abstractmethod

In [2]:
# initialize the ChatOllama model with desired parameters
# https://pypi.org/project/langchain-ollama/
# https://python.langchain.com/docs/integrations/chat/ollama/
# https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html
llm = ChatOllama(model="qwen2.5:7b", format="json")

In [3]:
# clip the history to the last 8000 characters
def clip_history(histroy: str, max_chars: int = 8000) -> str:
    if len(histroy) > max_chars:
        return histroy[-max_chars:]
    return histroy

In [4]:
# state
class TrpgState(TypedDict):
    history: str
    need_roll: bool
    roll_number: int

In [5]:
# base class of agents
class AgentBase(ABC):
    def __init__(self, state: TrpgState):
        self.state = state

    @abstractmethod
    def get_prompt_template(self) -> str:
        pass

    def execute(self) -> TrpgState:
        # clip history if too long
        self.state['history'] = clip_history(self.state['history'])
        # define prompt template
        template = self.get_prompt_template()
        prompt = PromptTemplate.from_template(template)
        # define and invoke llm chain
        llm_chain = prompt | llm | StrOutputParser()
        generation = llm_chain.invoke({
            'history': self.state['history'],
            'roll_number': self.state['roll_number']
        })
        # parse result
        data = json.loads(generation)
        self.state['need_roll'] = data.get('need_roll', False)
        self.state['roll_number'] = -1
        # append new history
        self.state['history'] += '\n' + generation
        self.state['history'] = clip_history(self.state['history'])
        return self.state

In [6]:
# Dungeon Master (DM)
class DM(AgentBase):
    tag = 'dungeon_master'
    
    def get_prompt_template(self) -> str:
        return """
            {history}
            As DnD DM, describe the current scenario for the player.
            In the first round, I do not need to roll the dice.
            When the player specifies an action, decide whether I need to throw a dice, and if a dice is rolled what will happen based on the rolling result.
            After rolling the dice for the player (i.e., the history ends with a dice rolling number), there is no need to roll a second time.
            The current rolling result is {roll_number}, if roll_number > 0, I need to explain how the roll affects the result (e.g., According to dice rolling, ...).
            Output in strict JSON format with no extra spaces, the JSON format is: {{'scenario': 'your action description', 'need_roll': True/False}}
        """

    def dm_node(state: TrpgState) -> TrpgState:
        print('> DM is planning...')
        return DM(state).execute()

In [7]:
# Player
class Player(AgentBase):
    tag = 'player'
    
    def get_prompt_template(self) -> str:
        return """
            As a Player from a DnD game, I need to specify an action every round according to the given scenario from the DM.
            I need to follow the current scenario and give different actions that make sense.
            Here is the interaction history (scenarios are from DM and actions are from me, the last scenario is the current scenario): {history}
            Output in strict JSON format with no extra spaces, the JSON format is: {{'action': 'my decided action (e.g., I want to ...)'}}
        """

    def player_node(state: TrpgState) -> TrpgState:
        print('> Player is thinking...')
        return Player(state).execute()

Now we define a tool - a function to roll the dice.

In [8]:
# dice rolling tool
DICE_NODE_NAME = 'dice'
def DiceRolling(state: TrpgState) -> TrpgState:
    print('> Rolling the dice...')
    random_num = random.randint(1, 20)
    state['history'] += f'\nroll result:{random_num}'
    state['history'] = clip_history(state['history'])
    state['need_roll'] = False  # reset
    state['roll_number'] = random_num
    return state

Then we define a conditional edge to decide which node to go to for dice rolling logic.

In [9]:
# conditional edge - check whether dice rolling is needed
def check_need_roll(state: TrpgState):
    if state.get('need_roll'):
        print('> Need to roll the dice.')
        return DICE_NODE_NAME
    else:
        # no need to roll, go back to player
        return Player.tag

Now we create a State Graph.

In [10]:
# create a graph where each function is a node
workflow = StateGraph(TrpgState)
workflow.add_node(DM.tag, DM.dm_node)
workflow.add_node(Player.tag, Player.player_node)
workflow.add_node(DICE_NODE_NAME, DiceRolling)
workflow.set_entry_point(DM.tag)  # starts from the dungeon master
workflow.add_conditional_edges(DM.tag, check_need_roll)  # DM decides whether to roll a dice
workflow.add_edge(Player.tag, DM.tag)  # Player tells DM what he wants to do
_ = workflow.add_edge(DICE_NODE_NAME, DM.tag)  # dice rolling result passed to DM to describe what will happen

Now we test~

In [11]:
# compile the workflow
app = workflow.compile()

In [12]:
# number of rounds to play
rounds = 3
# initial state: since the player is not involved yet, 
# no need to roll the dice and no former dice rolling result
initial_state = TrpgState(
    history='A monster appears in front of you.',
    need_roll=False,
    roll_number=-1
)

In [13]:
cur_round = 0
for s in app.stream(initial_state):
    if Player.tag in s:
        cur_round += 1
        print('-----------------------')
    cur_agent = list(s.keys())[-1]
    cur_state = list(s.values())[-1]
    print(f'> Round {cur_round} - {cur_agent}:')
    print(f'> Need to roll = {cur_state["need_roll"]}; Roll number = {cur_state["roll_number"]}')
    print('^^^^^^^^^^^^^^^^^^^^')
    print(f'> History: {cur_state["history"].strip()}')
    print('vvvvvvvvvvvvvvvvvvvv')
    if cur_round == rounds:
        print('> END')
        break

> DM is planning...
> Need to roll the dice.
> Round 0 - dungeon_master:
> Need to roll = True; Roll number = -1
^^^^^^^^^^^^^^^^^^^^
> History: A monster appears in front of you.
{"scenario":"A towering goblinoid monster steps into view, its body a twisted mass of green flesh and iron spikes. It snarls at you, revealing sharp fangs. The monster stands ready to attack.","need_roll":true}
vvvvvvvvvvvvvvvvvvvv
> Rolling the dice...
> Round 0 - dice:
> Need to roll = False; Roll number = 9
^^^^^^^^^^^^^^^^^^^^
> History: A monster appears in front of you.
{"scenario":"A towering goblinoid monster steps into view, its body a twisted mass of green flesh and iron spikes. It snarls at you, revealing sharp fangs. The monster stands ready to attack.","need_roll":true}
roll result:9
vvvvvvvvvvvvvvvvvvvv
> DM is planning...
> Round 0 - dungeon_master:
> Need to roll = False; Roll number = -1
^^^^^^^^^^^^^^^^^^^^
> History: A monster appears in front of you.
{"scenario":"A towering goblinoid monst

## Another Example About Tools

[Another demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/06%20Agent%20Choose%20Tool) here describes a scenario where a chat agent serves as an orchestrator to decide whether or not to use tools to answer the user's question. If the chat agent decides to use tools, he will forward the request to the tool agent to further decide which specific tool to use and what parameters to pass.

The implementation logic is pretty much similar to the DnD one, except that here we define an annotation method `@tool` to automatically manage a tool base for the agents. We will not demonstrate here.